This notebook will be used for Capstone Project from IBM Data Science Certification.

In [2]:
import pandas as pd
import numpy as np

In [3]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [4]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

In [5]:
df = df[df.Borough != 'Not assigned'].reset_index(drop=True)
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [6]:
df_group = df.groupby(['Postal Code','Borough'], as_index=False).agg(lambda x: ','.join(x))

In [7]:
na = df_group['Neighbourhood'] == "Not assigned"
df_group.loc[na, 'Neighbourhood'] = df_group.loc[na, 'Borough']
df_group.shape

(103, 3)

In [8]:
df_coord = pd.read_csv("http://cocl.us/Geospatial_data")

In [11]:
df_group = pd.merge(df_group, df_coord, on = 'Postal Code', how='inner')
df_group.head(12)

,Postal Code,Borough,Neighbourhood,Latitude_x,Longitude_x,Latitude_y,Longitude_y,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,43.806686,-79.194353,43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,43.784535,-79.160497,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,43.763573,-79.188711,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917,43.770992,-79.216917,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,43.773136,-79.239476,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,43.744734,-79.239476,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029,43.727929,-79.262029,43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577,43.711112,-79.284577,43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476,43.716316,-79.239476,43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,43.692657,-79.264848,43.692657,-79.264848
